## 펑션 콜링(function calling) 작동 방식 이해하기 

In [26]:
api_key = "your-api-key-here"

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm=OpenAI(model="gpt-4o-mini", api_key=api_key)

In [3]:
def add(a,b):
    """ 주어진 두 숫자를 더하고 결과를 출력합니다 """
    return a+b


def mul(a,b):
    """ 주어진 두 숫자를 곱하고 결과를 출력합니다 """
    return a*b

def div(a,b):
    """ 주어진 두 숫자 중 첫번 째 숫자를 두번 째 숫자로 나누고 결과를 출력합니다 """
    return a/b


In [4]:
at=FunctionTool.from_defaults(fn=add)
mt=FunctionTool.from_defaults(fn=mul)
dt=FunctionTool.from_defaults(fn=div)

In [5]:
agent_worker=ReActAgent(
                      tools=[at,mt,dt],
                      llm=llm,
                      verbose=False
                      )


In [6]:
response= await agent_worker.run("(77*2+2)를 78로 나눈 값을 계산해 줘")
print(response)

(77 * 2 + 2)를 78로 나눈 값은 2.0입니다.


## 펑션 콜링 실습 -외부 API를 활용하는 Function Calling Agent

기존 FunctionCalling Agent가 더 이상 지원을 하지않고 삭제되어 이후 코드는 ReAct Agent 코드로 진행합니다.

In [7]:
!pip install pandas-datareader llama-index

import pandas_datareader as pdr
from datetime import datetime, timedelta
import json
from typing import Dict, Optional
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import ReActAgent
from llama_index.llms.openai import OpenAI
import numpy as np

In [8]:
llm=OpenAI(model="gpt-4o-mini", api_key=api_key)

In [9]:
# 미국 주식 최신 종가 호출 
def get_stock_price_us(ticker: str) -> str:
    try:
        # 티커를 대문자로 변환하고 공백 제거
        ticker = ticker.upper().strip()
        
        # 기본적인 티커 형식 검증 (1-5자리 알파벳 + 선택적 하이픈/점)
        if not ticker or len(ticker) > 6:
            return f"'{ticker}'는 유효한 티커가 아닙니다. 올바른 티커 심볼을 입력하세요."
        
        # 5일치 데이터 가져오기
        end_date = datetime.now()
        start_date = end_date - timedelta(days=7)  # 주말 고려하여 7일
        
        # pandas-datareader로 Stooq에서 데이터 조회
        df = pdr.get_data_stooq(ticker, start=start_date, end=end_date)
        
        if df.empty:
            return f"{ticker}: 데이터를 찾을 수 없습니다. 티커를 확인하세요."
        
        # 최신 종가
        latest_price = round(df['Close'].iloc[-1], 2)
        latest_date = df.index[-1].strftime('%Y-%m-%d')
        
        # 전일 대비 변동 계산
        if len(df) > 1:
            prev_close = df['Close'].iloc[-2]
            change = latest_price - prev_close
            change_pct = (change / prev_close) * 100
            
            # 변동 표시
            if change > 0:
                change_str = f"+${abs(change):.2f} (+{change_pct:.2f}%)"
            elif change < 0:
                change_str = f"-${abs(change):.2f} ({change_pct:.2f}%)"
            else:
                change_str = "➡️ 변동 없음"
        else:
            change_str = "변동 정보 없음"
        
        return (f"{ticker}\n"
                f"현재가: ${latest_price:,.2f}\n"
                f"변동: {change_str}\n"
                f"기준일: {latest_date}")
        
    except Exception as e:
        # 에러 처리
        error_msg = str(e)
        if "No data fetched" in error_msg:
            return f"{ticker}: 존재하지 않는 티커이거나 상장폐지된 종목입니다."
        else:
            return f"{ticker} 조회 중 오류 발생: {error_msg[:100]}"

In [10]:
# FunctionTool 생성
stock_tool = FunctionTool.from_defaults(
    fn=get_stock_price_us,
    name="get_stock_price",
    description="""미국 주식 가격을 티커 심볼로 조회합니다.
    
    티커 심볼을 입력하세요:
    - AAPL (Apple)
    - TSLA (Tesla)  
    - GOOGL (Google/Alphabet)
    - MSFT (Microsoft)
    - AMZN (Amazon)
    - META (Meta/Facebook)
    - NVDA (Nvidia)
    - NFLX (Netflix)
    - 기타 모든 미국 주식 티커
    
    주의: 회사명이 아닌 티커 심볼을 입력해야 합니다."""
)

In [11]:
# Agent 생성
agent = ReActAgent(
    tools=[stock_tool],
    llm=llm,
    verbose=False,
    max_iterations=3,
    system_prompt="""You are a US stock market assistant.
    When users ask about stock prices by company name, you should:
    1. Convert the company name to its ticker symbol
    2. Use the get_stock_price tool with the ticker
    
    Common mappings:
    - Apple -> AAPL
    - Tesla -> TSLA
    - Google/Alphabet -> GOOGL
    - Microsoft -> MSFT
    - Amazon -> AMZN
    - Meta/Facebook -> META
    - Nvidia -> NVDA
    
    The tool requires ticker symbols, not company names."""
)

In [12]:
# 직접 함수 호출
print(get_stock_price_us("AAPL"))

# Agent 사용
response = await agent.run("테슬라 주가 알려줘")
print(response)

AAPL
현재가: $237.88
변동: +$3.53 (+1.51%)
기준일: 2025-09-08
테슬라(TSLA)의 현재 주가는 $346.40이며, 변동은 -$0.57 (-0.16%)입니다.


## 펑션 콜링 실습 - Context-Augmented Function Calling Agent

기존 FunctionCalling Agent가 더 이상 지원을 하지않고 삭제되어 이후 코드는 ReAct Agent 코드로 진행합니다.

In [13]:
#데이터 다운로드 
import os
import urllib.parse
import requests
import re

urls=[
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2022.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2023.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2024.pdf"
    ]

# 각 파일 다운로드
for url in urls:
    encoded_filename = url.split("/")[-1]  # URL에서 파일명 추출
    decoded_filename = urllib.parse.unquote(encoded_filename) # 한글 파일명 복원
    response = requests.get(url)
    if response.status_code == 200:
# 임시 파일명으로 저장
        temp_filename = "temp_download_file" + os.path.splitext(decoded_filename)[1] 
        with open(temp_filename, 'wb') as f:
            f.write(response.content)            
        os.rename(temp_filename, decoded_filename) # 파일명 변경
        print(f"완료: {decoded_filename} 다운로드 완료")
    else:
        print(f"오류: {url} 다운로드 실패 (상태 코드: {response.status_code})\n")


완료: [삼성전자]사업보고서_2022.pdf 다운로드 완료
완료: [삼성전자]사업보고서_2023.pdf 다운로드 완료
완료: [삼성전자]사업보고서_2024.pdf 다운로드 완료


In [14]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings

Settings.llm = OpenAI(model="gpt-4o-mini", api_key=api_key)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", api_key=api_key)

In [15]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex  
)
from llama_index.core.tools import QueryEngineTool

In [16]:
# load data
S2024_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2024.pdf"]
    ).load_data()
S2023_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2023.pdf"]
    ).load_data()
S2022_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2022.pdf"]
    ).load_data()

# build index
S2024_index = VectorStoreIndex.from_documents(S2024_docs)
S2023_index = VectorStoreIndex.from_documents(S2023_docs)
S2022_index = VectorStoreIndex.from_documents(S2022_docs)
    

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [17]:
S2024_engine = S2024_index.as_query_engine(similarity_top_k=3)
S2023_engine = S2023_index.as_query_engine(similarity_top_k=3)
S2022_engine = S2022_index.as_query_engine(similarity_top_k=3)

In [18]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=S2024_engine,
        name="samsung_2024",
        description=(
            "삼성전자의 2024년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2023_engine,
        name="samsung_2023",
        description=(
             "삼성전자의 2023년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2022_engine,
        name="samsung_2022",
        description=(
             "삼성전자의 2022년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
]

In [22]:
# agent 생성
agent = ReActAgent(
    tools=query_engine_tools,  # tools 대신 query_engine_tools 사용
    llm=llm,
    verbose=False,
    )


In [23]:
response = await agent.run(
    "삼성전자의 2022년, 2023년, 2024년 연결기준 매출액을 각각 알려주세요. "
    "각 연도의 매출액을 원화 단위로 제시해 주세요."
)
print(response)

삼성전자의 2022년 연결기준 매출액은 302조 2,314억원, 2023년 연결기준 매출액은 258조 9,355억원, 2024년 연결기준 매출액은 301조원입니다.


In [24]:
response= await agent.run("삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘")
print(response)

삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익은 다음과 같습니다:
- **2022년**: 매출액 302조 2,314억원, 영업이익 51조 6,633억원
- **2023년**: 매출액 258조 9,355억원, 영업이익 6조 5,669억원
- **2024년**: 매출액 301조 원, 영업이익 33조 원

2023년 매출액이 줄어든 이유는 글로벌 경제 상황, 공급망 문제, 시장 경쟁 심화 등이 있으며, 2024년에 회복된 이유는 경제 상황 개선, 공급망 회복, 신제품 출시 등이 있을 것으로 분석됩니다.


In [25]:
response=await agent.run("삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는 다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘")
print(response)

2024년 삼성전자의 매출 회복은 여러 요인에 기인합니다. 전체 매출이 전년 대비 16.2% 증가한 점에서 볼 수 있듯이, 다양한 제품 라인업의 성장이 중요한 역할을 했습니다. 특히, DS 부문에서 DRAM과 NAND Flash 등 고부가가치 제품의 수요가 크게 증가하여 66.8%의 매출 성장을 기록했습니다. 

또한, DX 부문에서도 소폭의 증가가 있었으며, 이는 스마트폰, TV, 냉장고 등 다양한 소비자 가전 제품의 판매 호조에 기인합니다. AI 기술을 접목한 Galaxy S24 시리즈와 Galaxy Z 폴드6 및 Z 플립6의 출시도 소비자들의 관심을 끌어 매출 증가에 기여했습니다.

마지막으로, 견실한 재무 구조와 브랜드 가치의 상승도 매출 회복에 긍정적인 영향을 미쳤습니다. 이러한 요소들이 결합되어 삼성전자는 2024년에 매출을 효과적으로 회복할 수 있었습니다.
